# Annotated_E3WS_rt.ipynb
**:auth:** Nathan T Stevens  
**:email:** ntsteven (at) uw.edu  
**:org:** Pacific Northwest Seismic Network    
**:license:**   
    Creative Commons Attribution 4.0 International (inherited from E3WS repository license)  
    (CC4.0-BY License)  
**:Purpose:**  
This notebook provides an annotated version of the example E3WS_rt.py script provided by Lara et al. (2023)  

**:Attribution:**  
This code modifies the code base from **E3WS, Pablo Lara et al. 2023, ** [![DOI](https://zenodo.org/badge/637827897.svg)](https://zenodo.org/badge/latestdoi/637827897)

And their paper:   

**Pablo Lara, Quentin Bletery, Jean-Paul Ampuero, Adolfo Inza, Hernando Tavera. Earthquake Early Warning Starting From 3 s of Records on a Single Station With Machine Learning. Journal of Geophysical Research: Solid Earth.**

**E3WS article: https://doi.org/10.1029/2023JB026575**

Import depdendences and do a bit of path definition work to re-define directory structure mapping

In [4]:
import os
import sys
import joblib
import collections
import datetime
import collections
import numpy as np
from obspy import read, UTCDateTime, Stream

# Local E3WS Path Mapping
E3WS_ROOT = os.path.join("..", "E3WS")
RT_ROOT = os.path.join(E3WS_ROOT, "real_time")
sys.path.append(RT_ROOT)
sys.path.append(os.path.join(RT_ROOT, "functions"))
# E3WS Modules
import pb_functions as pbi
import pb_utils_v16 as pb_utils
import pick_func_v16 as pick_func
from utils import pb_SAM

Set a range of environmental parameters (need to decode these a bit...)

In [5]:
# Software parameters
flag_plot = 1  # plot detection, P-phase picking and first magnitude estimation
flag_writefile = 1  # write logs
flag_trackmag = True
pb_version = 16
pb_subversion = 0
n_models = 5  # number of models for tracking source characterization, 1: just 3 sec model, 58: 3 sec to 60 sec models
pb_inst = True  # True for instrument correction, you have to add pz/ folder with pzfile

# Model parameters
Ptrain = 0.5  # Dont change
mov_time = 0.1  # moving time for estimate P-phase arrival time
thr = 0.80  # detection threshold to trigger an event
# For BAZ models
n_models_baz = 1  # Predict just using 3 s of P-wave


# Some more path defining
# folder_models = "../models/"
folder_models = os.path.join(E3WS_ROOT, "models")
# folder_out = 'results'
folder_out = os.path.join(RT_ROOT, "results")



Load station metadata and Poles and Zeros (instrument response)  
*TODO: Add option for attached responses in this workflow*  
  
AND  
  
Set up results logging file  

In [7]:
# Station parameters
station = "SLRZ"
sta_lat = -12.074200
sta_lon = -77.233200

if pb_inst == True:
    print("Using Pole Zero")
    # pzfile = "pz/" + station + "_HN.pz"
    pzfile = os.path.join(RT_ROOT,'pz',station,'_HN.pz')
else:
    print("NOT using Pole Zero")
    pzfile = None

print("PZFILE:", pzfile)

#Earthquake name
eq_name = 'Canta_M5.6_20220107'

# Try to create output folder if model logging is on
if flag_writefile == 1:
    try:
        os.makedirs(folder_out)
    except Exception:
        print("Folder out already exist")

    # Write headers to log file
    f = open(os.path.join(folder_out,f'{station:s}_{eq_name:s}.csv'), "w")
    f.write(
        "P_AI_date,starttime,endtime,P-wave(s),mag_pred(M),lat_pred,lon_pred,dis_pred(km),dep_pred(km),baz_pred(°)\n"
    )
    f.close()

Using Pole Zero
PZFILE: ../E3WS/real_time/pz/SLRZ/_HN.pz
Folder out already exist


## Load data


In [13]:
# Waveform file
# org_time = obspy.UTCDateTime('2022-01-07T10:27:06.30')#By IGP, just for trim
file_E = os.path.join(RT_ROOT, "data", eq_name, f"{station}.HNE.PE.2022.007.mseed")
file_N = os.path.join(RT_ROOT, "data", eq_name, f"{station}.HNN.PE.2022.007.mseed")
file_Z = os.path.join(RT_ROOT, "data", eq_name, f"{station}.HNZ.PE.2022.007.mseed")
format = "MSEED"

# Read waveforms
st_raw = Stream()
for _f in [file_E, file_N, file_Z]:
    st_raw += read(_f)
# data = "data/" + eq_name + "/"

# st_raw = read(data + file_E, format=format)
# st_raw += read(data + file_N, format=format)
# st_raw += read(data + file_Z, format=format)

# Print raw data
print(st_raw)

3 Trace(s) in Stream:
PE.SLRZ.14.HNE | 2022-01-07T10:27:01.300000Z - 2022-01-07T10:29:06.300000Z | 100.0 Hz, 12501 samples
PE.SLRZ.14.HNN | 2022-01-07T10:27:01.300000Z - 2022-01-07T10:29:06.300000Z | 100.0 Hz, 12501 samples
PE.SLRZ.14.HNZ | 2022-01-07T10:27:01.300000Z - 2022-01-07T10:29:06.300000Z | 100.0 Hz, 12501 samples


## Load Models

In [17]:
print("Reading models")
if flag_trackmag == 1:
    # n_models = 1 #58 in total, 1 model, 1 estimation
    MAG_md = [
        "MAG_Whole_StackingXGB_7tp"
        + str(i)
        + "f45_v"
        + str(pb_version)
        + "."
        + str(pb_subversion)
        + ".joblib"
        for i in np.arange(n_models) + 3
    ]
    DIS_md = [
        "DIS_Whole_StackingXGB_7tp"
        + str(i)
        + "f45_v"
        + str(pb_version)
        + "."
        + str(pb_subversion)
        + ".joblib"
        for i in np.arange(n_models) + 3
    ]
    DEP_md = [
        "DEP_Whole_StackingXGB_7tp"
        + str(i)
        + "f45_v"
        + str(pb_version)
        + "."
        + str(pb_subversion)
        + ".joblib"
        for i in np.arange(n_models) + 3
    ]
    BAZ_cos_md = [
        "welloriented_BAZ_Cos_STEAD_StackingXGB_7tp"
        + str(i)
        + "f45_v"
        + str(pb_version)
        + "."
        + str(pb_subversion)
        + ".joblib"
        for i in np.arange(n_models_baz) + 3
    ]
    BAZ_sin_md = [
        "welloriented_BAZ_Sin_STEAD_StackingXGB_7tp"
        + str(i)
        + "f45_v"
        + str(pb_version)
        + "."
        + str(pb_subversion)
        + ".joblib"
        for i in np.arange(n_models_baz) + 3
    ]

    # print files:
    print(MAG_md)

    MAG_pb = [
        joblib.load(open(os.path.join(E3WS_ROOT,'models',"MAG",MAG_md[i]), "rb"))
        for i in range(n_models)
    ]
    DIS_pb = [
        joblib.load(open(os.path.join(E3WS_ROOT,'models',"DIS",DIS_md[i]), "rb"))
        for i in range(n_models)
    ]
    DEP_pb = [
        joblib.load(open(os.path.join(E3WS_ROOT,'models',"DEP",DEP_md[i]), "rb"))
        for i in range(n_models)
    ]
    BAZ_cos_pb = [
        joblib.load(open(os.path.join(E3WS_ROOT,'models',"BAZ",BAZ_cos_md[i]), "rb"))
        for i in range(n_models_baz)
    ]
    BAZ_sin_pb = [
        joblib.load(open(os.path.join(E3WS_ROOT,'models',"BAZ",BAZ_sin_md[i]), "rb"))
        for i in range(n_models_baz)
    ]


# Reading DET and PICK model
DET_pb = joblib.load(
    open(
        os.path.join(E3WS_ROOT,'models','DET',"DET_"
        + station
        + "_Whole_XGB_10tp0.5to4.0each0.5f7_unbalanced_official_v16.joblib"),
        "rb",
    )
)
PICK_pb = joblib.load(
    open(
        os.path.join(E3WS_ROOT,'models','PICK','PICK_CLF_XGBdep4n6000bytree0.46level1.0node1.0_P0.5_NPS_eachpoint_spike_v16.joblib'),
        "rb",
    )
)

print("Reading models complete")

Reading models
['MAG_Whole_StackingXGB_7tp3f45_v16.0.joblib', 'MAG_Whole_StackingXGB_7tp4f45_v16.0.joblib', 'MAG_Whole_StackingXGB_7tp5f45_v16.0.joblib', 'MAG_Whole_StackingXGB_7tp6f45_v16.0.joblib', 'MAG_Whole_StackingXGB_7tp7f45_v16.0.joblib']
Reading models complete


In [87]:
# Look at properties of some of these model objects
print(MAG_pb)
_tmp = MAG_pb[0]
print(_tmp)
_tmp = _tmp.base_models_[0]

_tmp = _tmp[0]
print(_tmp.get_params().keys())
# _tmp = _tmp[0].best_estimator_
# # print(_tmp)
# _tmp = _tmp[1]
# print(_tmp)
# print(_tmp)

[<utils.pb_SAM object at 0x2a8215ee0>, <utils.pb_SAM object at 0x2a825ad00>, <utils.pb_SAM object at 0x2ac4aefd0>, <utils.pb_SAM object at 0x2af3b8eb0>, <utils.pb_SAM object at 0x2ac4affa0>]
dict_keys(['memory', 'steps', 'verbose', 'standardscaler', 'xgbregressor', 'standardscaler__copy', 'standardscaler__with_mean', 'standardscaler__with_std', 'xgbregressor__objective', 'xgbregressor__base_score', 'xgbregressor__booster', 'xgbregressor__callbacks', 'xgbregressor__colsample_bylevel', 'xgbregressor__colsample_bynode', 'xgbregressor__colsample_bytree', 'xgbregressor__early_stopping_rounds', 'xgbregressor__enable_categorical', 'xgbregressor__eval_metric', 'xgbregressor__gamma', 'xgbregressor__gpu_id', 'xgbregressor__grow_policy', 'xgbregressor__importance_type', 'xgbregressor__interaction_constraints', 'xgbregressor__learning_rate', 'xgbregressor__max_bin', 'xgbregressor__max_cat_to_onehot', 'xgbregressor__max_delta_step', 'xgbregressor__max_depth', 'xgbregressor__max_leaves', 'xgbregress

In [21]:
# Select ENZ
st_e = st_raw.select(component="E")
st_n = st_raw.select(component="N")
st_z = st_raw.select(component="Z")

# Quality control of traces
n_traces = min(len(st_e), len(st_n), len(st_z))

display(n_traces)



1

## Main Loop


In [ ]:
# Iterate across number of traces in stream - this is quite a fragile way of handling multiple traces...
for i in range(0, n_traces):
    print(i, n_traces)
    # Order components as ENZ (reverse of most picker routines, but typical alphabetical order of others)
    st = Stream()
    st = st.append(st_e[i])
    st = st.append(st_n[i])
    st = st.append(st_z[i])

    # Do maximum complete length trimming
    maxstart = np.max([tr.stats.starttime for tr in st])
    minend = np.min([tr.stats.endtime for tr in st])
    st.trim(maxstart, minend)

    # Create copy of initial data (is this strictly necessary?)
    st_process = st.copy()

    # Get sampling rate
    Fs = st_process[0].stats.sampling_rate
    # TODO: Shift this to processing parameters - unnecessary
    slidding_seconds = 1
    # Get number of data in trace
    N = len(st_process[0].data)
    # Get step size
    w_sec = 10
    # Calculate number of windows
    n_waves = int((N / Fs - w_sec) / slidding_seconds) + 1  # slidding 1s

    # Define holder arrays - TODO: this can be improved by preallocating space using the metrics calculated above
    PROB_P = np.array([])
    PROB_N = np.array([])
    PROB_S = np.array([])
    PROB_PP = np.array([])
    
    # TODO: Remind myself of what the `collections.deque` method does...
    buffer_proba = collections.deque(maxlen=5)
    buffer_proba_n = collections.deque(maxlen=5)

    # Set up counters?
    eq_thr_past = 0
    eq_thr_present = 0

    flag_magnitude = 0
    c_mag = 0  # counter magnitude (1 times)
    waiting_sec = 0
    n_eq = 0

    # Define another holder array... probably shift this up to where the other holders are instantiated?
    DL = np.array([])

    # Iterate across windows - TODO: this can be DRASITCALLY simplified with a st.slide()?
    for i3 in range(0, n_waves):
        # Create ANOTHER COPY?
        st_pb = st_process.copy()
        # Get window starttime
        t1 = i3 * slidding_seconds
        # Get window endtime
        t2 = w_sec + t1
        # Trim window
        st_pb = st_pb.trim(st_pb[0].stats.starttime + t1, st_pb[0].stats.starttime + t2)

        ### CALCULATE FEATURE VECTOR INCLUDING PRE-PROCESSING
        # A lot of this pre-processing can be done before windowing data to save repeat calculations
        # Feature vector
        FV = pb_utils.st_FV(st_pb, pb_inst, pzfile=pzfile, fmin=1.0, fmax=7.0)
        FV = np.real([FV])

        ### RUN DETECTION MODEL
        # Get probabilities
        prob_nps = DET_pb.predict_proba(FV)[0]
        # Extract modeled values
        prob_n = prob_nps[0]  # v8 - noise probability
        prob_p = prob_nps[1]  # v8 - p-wave probability
        prob_s = prob_nps[2]  # v8 - s-wave probability
        # Output result (`black` code formatting)
        print(
            i3,
            round(t2, 3),
            st_pb[0].stats.starttime,
            st_pb[0].stats.endtime,
            len(st_pb[0].data),
            prob_n,
            prob_p,
            prob_s,
            n_waves,
        )
        # Save probabilities to holder numpy arrays (again, this can be accelerated by preallocating space)
        PROB_N = np.append(PROB_N, prob_n)
        PROB_P = np.append(PROB_P, prob_p)
        PROB_S = np.append(PROB_S, prob_s)

        ### write probabilities to collections that serve as sliding (read-write buffers?)
        # Circular vector probability
        buffer_proba.append(prob_p) # append prob_p to right side of p-probabilitiy deque ('deck')
        buffer_proba_np = np.array(buffer_proba) # Take the list-like "collection", convert to np array, and 
        buffer_proba_np = np.flip(buffer_proba_np)  # First element is the current proba
        breakpoint()
        buffer_proba_n.append(prob_n)  # Prob of being N
        buffer_proba_of = np.array(buffer_proba_n)
        buffer_proba_of = np.flip(buffer_proba_of)  # First element is the current proba

        ## TODO: Logic for this trigger can probably be cleaned up with obspy.signal.trigger_onset?
        # Logic for trigger

        # If the 3-element mean probability surpasses a threshold and there are 3+ data to work with, start triggering process
        if np.mean(buffer_proba_np[0:3]) >= thr and len(buffer_proba_np) >= 3:
            eq_thr_present = 1

        # If trigger is ON, do sanity check if high probability noise classification negates this
        if eq_thr_present == 1:
            # If there is a high probability of being noise, reject earlier trigger on
            if np.mean(buffer_proba_of) >= 0.99:
                eq_thr_present = 0

        # If this is INDEED a new event, run pick prediction
        if eq_thr_past == 0 and eq_thr_present == 1:
            print("New event!:")

            # ANOTHER FIXED PARAMETER?? - TODO: Move this up to a parameter_control block...?
            w_sec_pick = 8
            st_pick = st_pb.copy().trim(
                st_pb[0].stats.endtime - w_sec_pick, st_pb[0].stats.endtime
            )

            ## Run P-pick prediction - AGAIN , LOTS OF REPEATED PRE-PROCESSING
            PROB_PP = pick_func.PP_pick(
                st_pick, mov_time, pb_inst, pzfile, PICK_pb, fmin=1.0, fmax=7.0
            )  # SASPe for use pzfile, STEAD for not


            # Take peak probability value as P-pick, regardless of probability amplitude....
            P_AI_date = (
                st_pick[0].stats.starttime + np.argmax(PROB_PP) * mov_time + 4 - Ptrain
            )
            print("P arrival at:", P_AI_date)  # Must delete

            # If saving data for plotting, dump a bunch of stuff to holder arrays.
            if flag_plot == 1:
                REF = np.append(REF, st_pick[0].stats.starttime - st[0].stats.starttime)
                if len(REF) == 1:
                    PROB_PP_TENSOR = np.array([PROB_PP])
                    P_AI_DATE_VECTOR = [str(P_AI_date)]
                else:
                    PROB_PP_TENSOR = np.vstack((PROB_PP_TENSOR, PROB_PP))
                    P_AI_DATE_VECTOR.append(str(P_AI_date))

            # Switch on magnitude calculation 
            # Calculate magnitude
            flag_magnitude = 1  # Change 1 to calculate source parameters

        # TODO: Change a lot of these flag_* things to type BOOL...
        ## THIS IS THE GUTS OF WHAT WE WANT TO PLAY WITH
        if flag_magnitude == 1:
            print("MAG_time:", st_pb[0].stats.endtime - P_AI_date)
            # Wait until trace has minimum 3.0 sec
            if st_pb[0].stats.endtime - P_AI_date >= 3.0:
                st_pb_reg = st_process.copy().trim(P_AI_date - 7, P_AI_date + 3 + c_mag)

                # FV for REG and BAZ
                FV45 = pb_utils.st_FV(
                    st_pb_reg, pb_inst, pzfile=pzfile, fmin=1.0, fmax=45.0
                )
                FV45 = np.real([FV45])

                # Regression for MAG, DIS and DEP
                MAG_pb_predict = MAG_pb[c_mag].predict(FV45)[0]
                DIS_pb_predict = DIS_pb[c_mag].predict(FV45)[0]
                DEP_pb_predict = DEP_pb[c_mag].predict(FV45)[0]
                if c_mag <= n_models_baz - 1:
                    BAZ_pb_cos_predict = BAZ_cos_pb[c_mag].predict(FV45)[0]
                    BAZ_pb_sin_predict = BAZ_sin_pb[c_mag].predict(FV45)[0]

                    # Cos, Sin, to BAZ angle
                    BAZ_pb_predict = (
                        np.arctan2(BAZ_pb_sin_predict, BAZ_pb_cos_predict) * 180 / np.pi
                    )

                    # Get eq.lat, eq.lon
                    eq_lat, eq_lon = pb_utils.pb_getpoint(
                        sta_lat, sta_lon, DIS_pb_predict, BAZ_pb_predict
                    )
                    eq_lat = round(eq_lat, 6)
                    eq_lon = round(eq_lon, 6)

                print(
                    "-->",
                    P_AI_date,
                    st_pb_reg[0].stats.starttime,
                    st_pb_reg[0].stats.endtime,
                    st_pb_reg[0].stats.endtime - st_pb_reg[0].stats.starttime,
                    MAG_pb_predict,
                    eq_lat,
                    eq_lon,
                    DIS_pb_predict,
                    DEP_pb_predict,
                )

                if flag_trackmag == True and flag_writefile == 1:
                    f = open(folder_out + "/" + station + "_" + eq_name + ".csv", "a")
                    f.write(
                        str(P_AI_date)
                        + ","
                        + str(st_pb_reg[0].stats.starttime)
                        + ","
                        + str(st_pb_reg[0].stats.endtime)
                        + ","
                        + str(
                            st_pb_reg[0].stats.endtime
                            - st_pb_reg[0].stats.starttime
                            - 7
                        )
                        + ","
                        + str(MAG_pb_predict)
                        + ","
                        + str(eq_lat)
                        + ","
                        + str(eq_lon)
                        + ","
                        + str(DIS_pb_predict)
                        + ","
                        + str(DEP_pb_predict)
                        + ","
                        + str(BAZ_pb_predict)
                        + "\n"
                    )
                    f.close()

                # Save first magnitude estimation just for plot
                if c_mag == 0:
                    MAG_3s = MAG_pb_predict

                c_mag = c_mag + 1

        # computing magnitude c_mag times
        if c_mag == n_models:
            flag_magnitude = 0
            c_mag = 0

        # Save past values
        eq_thr_past = eq_thr_present